In [1]:
import nest_asyncio
import twint
from models import *
from pymongo import MongoClient
from pycoingecko import CoinGeckoAPI
from datetime import datetime, timezone

### Conectar con MongoDB

In [2]:
mongo = MongoClient('8.tcp.ngrok.io:11181')

db = mongo.proyectoBigData

### Inicializar CoinGecko API Wrapper

In [3]:
cg = CoinGeckoAPI()

### Recuperar id, nombre y abreviación de las criptomonedas disponibles

In [4]:
criptocurrency_list = cg.get_coins_list()

db.criptomonedas.insert_many(criptocurrency_list)

print(f'{len(criptocurrency_list)} Criptomoneadas registradas en DB')

7863 Criptomoneadas registradas en DB


### Recuperar historial de precios, capitalización de mercado y volumen de las criptomonedas seleccionadas

In [5]:
# Definimos los tokens a buscar
token_list = ["bitcoin", "dogecoin", "ethereum"]

# Fecha desde la que se comienza a buscar
from_date = datetime(2020, 1, 1)

days_ago = (datetime.now() - from_date).days

# intervalo entre cada recuperación de datos
time_interval = 'daily'

# moneda fíat con la que comparar el valor de las criptomonedas a buscar
moneda_comparacion = 'usd'

for token in token_list:
    
    # Recuperar historial de la criptomoneda
    historial = cg.get_coin_market_chart_by_id(id=token, vs_currency=moneda_comparacion,
                                               days=days_ago, interval=time_interval)
    
    # Histórico precios
    price_list = [vars(Precio(token, x[1], datetime.fromtimestamp(x[0] / 1000, timezone.utc)))
                  for x in  historial['prices']]
    
    db.priceHistory.insert_many(price_list)
    
    # Histórico de capitalización de mercado
    market_cap_list = [vars(MarketCap(token, x[1], datetime.fromtimestamp(x[0] / 1000, timezone.utc)))
                       for x in  historial['market_caps']]
    
    db.marketCapHistory.insert_many(market_cap_list)
    
    # Histórico de volumen de negociación
    volume_list = [vars(Volume(token, x[1], datetime.fromtimestamp(x[0] / 1000, timezone.utc)))
                   for x in  historial['total_volumes']]
    
    db.volumeHistory.insert_many(volume_list)

    print(f'{len(price_list)} registros históricos de {token} insertados en DB')

527 registros históricos de bitcoin insertados en DB
527 registros históricos de dogecoin insertados en DB
527 registros históricos de ethereum insertados en DB


### Recuperar tweets relacionados a criptomonedas

In [6]:
nest_asyncio.apply()

# Lista de palabras a buscar
keywords = [
        "cripto",
        "criptomoneda",
        "criptomonedas",
        "crypto",
        "cryptocurrency",
        "cryptocurrencies",
        "bitcoin",
        "BTC",
        "$BTC",
        "ethereum",
        "ETH",
        "$ETH",
        "dogecoin",
        "DOGE",
        "$DOGE",
    ]

# Inicializamos el scraper de Twitter
config = twint.Config()

# Excluimos retweets de la búsqueda
config.Retweets = False

# Mínimo de likes que deba tener un tweet para ser recuperado
config.Min_likes = 1000

# Fecha de inicio de recuperación de tweets
config.Since = '2020-01-01'

config.Hide_output = True
config.Store_object = True

lista_tweets = []

for word in keywords:

    # Seleccionamos la palabra clave a buscar
    config.Search = word
    
    # Realizamos la búsqueda
    twint.run.Search(config)
    
    # Recuperamos los resultados
    tweets = twint.output.tweets_list
    
    # Procesamos los resultados
    tweets_clean = [vars(Tweet(x.id, x.tweet, x.hashtags, x.cashtags,
                               datetime.strptime(x.datestamp + x.timestamp + x.timezone, '%Y-%m-%d%H:%M:%S%z'),
                               x.username, x.name, x.link, word, x.likes_count, x.retweets_count, x.replies_count)) for x in tweets]

    lista_tweets.extend(tweets_clean)

    # Añadimos a la lista de resultados
    print(f'{len(tweets_clean)} tweets con la keyword {word} registrados en DB')

# Guardamos en mongo
db.tweetsCripto.insert_many(lista_tweets)

print(f'{len(lista_tweets)} tweets registrados en DB')

2950 tweets con la keyword cripto registrados en DB
3011 tweets con la keyword criptomoneda registrados en DB
3053 tweets con la keyword criptomonedas registrados en DB
5988 tweets con la keyword crypto registrados en DB
6568 tweets con la keyword cryptocurrency registrados en DB
6701 tweets con la keyword cryptocurrencies registrados en DB
15874 tweets con la keyword bitcoin registrados en DB
17354 tweets con la keyword BTC registrados en DB
17892 tweets con la keyword $BTC registrados en DB
18398 tweets con la keyword ethereum registrados en DB
19305 tweets con la keyword ETH registrados en DB
19853 tweets con la keyword $ETH registrados en DB
21741 tweets con la keyword dogecoin registrados en DB
23525 tweets con la keyword DOGE registrados en DB
24149 tweets con la keyword $DOGE registrados en DB
206362 tweets registrados en DB


### Query en MongoDB para obtener tweets sin repeticiones

```
db.tweetsCripto.aggregate([{
    $group: {
        _id: {
            id: "$id"
        },
        cashtag: {
            $first: "$cashtag"
        },
        fecha: {
            $first: "$fecha"
        },
        hashtag: {
            $first: "$hashtag"
        },
        likes: {
            $first: "$likes"
        },
        link: {
            $first: "$link"
        },
        name: {
            $first: "$name"
        },
        replies: {
            $first: "$replies"
        },
        retweets: {
            $first: "$retweets"
        },
        tweet: {
            $first: "$tweet"
        },
        user: {
            $first: "$user"
        },
        search_keywords: {
            $addToSet: "$search"
        },
        count: {
            $sum: 1
        }
    }
}])
```